In [1]:
%%javascript

IPython.tab_as_tab_everywhere = function(use_tabs) {
    if (use_tabs === undefined) {
        use_tabs = true; 
    }

    // apply setting to all current CodeMirror instances
    IPython.notebook.get_cells().map(
        function(c) {  return c.code_mirror.options.indentWithTabs=use_tabs;  }
    );
    // make sure new CodeMirror instances created in the future also use this setting
    CodeMirror.defaults.indentWithTabs=use_tabs;

    };

IPython.tab_as_tab_everywhere()

<IPython.core.display.Javascript object>

In [2]:
# Libraries
from os import listdir
import re

import pandas as pd
import igraph as ig
from collections import defaultdict

import pynauty as pynauty
import sympy.combinatorics as combinatorics

import sage.groups.perm_gps.permgroup as sage_permgroup
import sage.groups.perm_gps.permgroup_named as sage_named_groups

In [3]:
def get_pynauty_graph(ig_graph):
	# Represent igraph as adjacency list-of-dictionaries
	adjacency_dict = {}
	for source in ig_graph.vs.indices:
		adjacent_edges = ig_graph.es.select(_source = source)
		adjacent_nodes = []
		for edge in adjacent_edges:
			if(edge.source == source):
				adjacent_nodes.append(edge.target)
			else:
				adjacent_nodes.append(edge.source)
		adjacency_dict[source] = adjacent_nodes

	pynauty_graph = pynauty.Graph(number_of_vertices = ig_graph.vcount(),
								  directed = False,
								  adjacency_dict = adjacency_dict)
	
	return(pynauty_graph)

def get_sectors(permutations):
	permutation_domains = []
	for permutation in permutations:
		permutation_domains.append([value for sublist in permutation.cyclic_form for value in sublist])
		
	return(list(merge_common(permutation_domains)))

# this is the function taken from the internet, it would be better to re-write it
def merge_common(lists):
	neigh = defaultdict(set)
	visited = set()
	for each in lists:
		for item in each:
			neigh[item].update(each)
	def comp(node, neigh = neigh, visited = visited, vis = visited.add):
		nodes = set([node])
		next_node = nodes.pop
		while nodes:
			node = next_node()
			vis(node)
			nodes |= neigh[node] - visited
			yield node
	for node in neigh:
		if node not in visited:
			yield sorted(comp(node))

def get_permutations_on_sector_id(permutations, sectors, sector_id):
	is_permutation_on_sector = lambda permutation, sector_id : all([node in set(sectors[sector_id]) for node in permutation])

	permutations_to_return = []
	for permutation in permutations:
		permutation_cyclic = [value for sublist in permutation.cyclic_form for value in sublist]
		if(is_permutation_on_sector(permutation_cyclic, sector_id)):
			permutations_to_return.append(permutation)
	return(permutations_to_return)

def sympy_permutations_to_sage_group(sympy_permutations):
	subgroup_perms_sage_form = []
	
	for i in range(len(sympy_permutations)):
		permutation = sympy_permutations[i].cyclic_form
		permutation_sage = []
		for j in range(len(permutation)):
			permutation_sage.append(tuple(permutation[j]))
		subgroup_perms_sage_form.append(permutation_sage)

	sage_group = sage_permgroup.PermutationGroup(subgroup_perms_sage_form)
	return(sage_group)

def classify_sympy_perm_group(sympy_permutations, domain_size):
	sage_group = sympy_permutations_to_sage_group(sympy_permutations)

	group_name = "Unknown"
	for i in range(1, domain_size + 1):
		if(sage_group.is_isomorphic(sage_named_groups.SymmetricGroup(i))):
			group_name = "S" + str(i)
			return(group_name)
	
	for i in range(1, domain_size + 1):
		if(sage_group.is_isomorphic(sage_named_groups.DihedralGroup(i))):
			group_name = "D" + str(i)
			return(group_name)

	for i in range(1, domain_size + 1):
		if(sage_group.is_isomorphic(sage_named_groups.CyclicPermutationGroup(i))):
			group_name = "C" + str(i)
			return(group_name)

	for i in range(1, domain_size + 1):
		if(sage_group.is_isomorphic(sage_named_groups.AlternatingGroup(i))):
			group_name = "A" + str(i)
			return(group_name)
		
	return(group_name)

def get_assigned_sectors(sectors, ig_graph):
	sectorId = []
	for i in ig_graph.vs.indices:
		search_result = [i in sector for sector in sectors]
		if (any(search_result) == False):
			sectorId.append(-1)
		else:
			sectorId.append([idx + 1 for idx in range(len(search_result)) if search_result[idx] == True][0])
	
	trivialIdx = max(sectorId) + 1
	for idx in range(len(sectorId)):
		if sectorId[idx] == -1:
			sectorId[idx] = trivialIdx
			trivialIdx = trivialIdx + 1
	return(sectorId)

In [4]:
def run_directory(path):
	files = listdir(path)
	
	nodefile = list(filter(re.compile("nodes.csv").search, files))[0]
	pattern = re.compile("_edges.csv")
	files = list(filter(pattern.search, files))
	
	old_nodes = pd.read_csv(path + nodefile, sep = ",")
	
	for file_idx in range(len(files)):
		edgelist = pd.read_csv(path + files[file_idx], sep = ",")
		ig_graph = ig.Graph.TupleList(edgelist.itertuples(index = False), directed = False)

		pynauty_graph = get_pynauty_graph(ig_graph)
		generators, grpsize1, grpsize2, orbits, numorbits = pynauty.autgrp(pynauty_graph)

		permutations = [combinatorics.Permutation(generator) for generator in generators]
		sectors = get_sectors(permutations)
		
		ncolors = re.sub("([0-9]*)_edges.csv$", "\\1", files[file_idx])
		nodes = pd.DataFrame(data = {"Label" : ig_graph.vs["name"],
									 "OrbitsNC" + ncolors : orbits,
									 "SectorNC" + ncolors : get_assigned_sectors(sectors, ig_graph)})
		nodes = nodes.sort_values(by = ["Label"], ignore_index = True)
		old_nodes = old_nodes.join(nodes[nodes.columns[1:3]])

		get_names_from_list_of_ids = lambda ig_graph, ids : [ig_graph.vs["name"][j] for j in ids]

		with open(re.sub("_edges.csv$", "_permutations.txt", path + files[file_idx]), 'w') as filehandle, open(re.sub("_edges.csv$", "_permutations_list.txt", path + files[file_idx]), 'w') as filehandle2:
			filehandle2.writelines("%s\n" % ig_graph.vs["name"])
			for i in range(len(sectors)):
				subgroup_perms = get_permutations_on_sector_id(permutations, sectors, i)
				for j in range(len(subgroup_perms)):
					filehandle2.writelines("%s\n" % subgroup_perms[j].array_form)
				filehandle2.writelines("\n")
				permutation_subgroup = combinatorics.PermutationGroup(subgroup_perms)

				# get subgroup orbits
				subgroup_orbits = [orbit for orbit in permutation_subgroup.orbits() if len(orbit) > 1]
				subgroup_orbits = [get_names_from_list_of_ids(ig_graph, list(orbit)) for orbit in subgroup_orbits]

				filehandle.writelines("Class: %s\n" % classify_sympy_perm_group(subgroup_perms, len(sectors[i])))
				filehandle.writelines("Sector:\n\t%s\n" % get_names_from_list_of_ids(ig_graph, sectors[i]))
				filehandle.writelines("Orbits (%i):\n" % len(subgroup_orbits))
				for i in range(len(subgroup_orbits)):
					filehandle.writelines("\t%i:\t%s\n" % (i, subgroup_orbits[i]))
				filehandle.writelines("Permutations:\n")

				for permutation in subgroup_perms:
					filehandle.writelines("\t%s\n" % [get_names_from_list_of_ids(ig_graph, nodes) for nodes in permutation.cyclic_form])
				filehandle.writelines("\n")

				#print(classify_sympy_perm_group(subgroup_perms, len(sectors[i])))
				#print(get_names_from_list_of_ids(ig_graph, sectors[i]))
				#print(subgroup_orbits)
				#for permutation in subgroup_perms:
				#	print([get_names_from_list_of_ids(ig_graph, nodes) for nodes in permutation.cyclic_form])
				#print("\n")

	old_nodes.to_csv(path + nodefile, index = False)

In [5]:
#run_directory("/home/ian/Dropbox (City College)/Research/PhD work/shared folders/PROTEIN-FOLDING/COLLABORATION-ANALYSIS/DAVID-FRANCESCO/VISUALIZATIONS/cluster/bw_no_sym/")

In [8]:
path = "/home/ian/Dropbox (City College)/Research/PhD work/shared folders/PROTEIN-FOLDING/COLLABORATION-ANALYSIS/DAVID-FRANCESCO/FINAL-ANALYSIS-MAY-2021/DATA/INTEGER-PROGRAM-COST-ONE-OVER-MAX-DEGREE/OUTPUT/"

dirs = listdir(path)

for dir in dirs:
	run_directory(path + dir + "/")

In [9]:
#permutation_group = combinatorics.PermutationGroup(permutations)
#print(permutation_group)
#print(permutation_group.orbits())

#group = sympy_permutations_to_sage_group(permutations)
#print(group)
#normal_subgroups = []
#for i in range(len(sectors)):
#	subgroup_permutations = get_permutations_on_sector_id(permutations, i)
#	normal_subgroup = sympy_permutations_to_sage_group(subgroup_permutations)
#	print(normal_subgroup)
#	print(normal_subgroup.is_normal(group))
#	normal_subgroups.append(normal_subgroup)

#print(normal_subgroups)

#if (len(normal_subgroups) > 1):
#	dir_product = normal_subgroups[0].direct_product(normal_subgroups[1], False)
#	for i in range(2, len(normal_subgroups)):
#		dir_product = dir_product.direct_product(normal_subgroups[i], False)
#else:
#	dir_product = normal_subgroups[0]

#print(dir_product)
#print(dir_product.is_isomorphic(group))